## Table of Contents

### 1. Importing libraries
### 2. Importing data
### 3. Data consistency checks - df_ords_prior
##### 1. Mixed-type values
##### 2. Missing values
##### 3. Duplicate values
### 4. Merging df_ords and df_ords_prior
### 5. Exporting df_merged_large

# 01. Importing libraries

In [1]:
# Import libraries
import pandas as pd
import numpy as np
import os

# 02. Importing data

In [2]:
# Create path shortcut
path = r'/Users/taraperrigeold/Documents/Documents - Tara Perrige’s MacBook Pro/CareerFoundry/Instacart Basket Analysis'

In [3]:
# Check the output
path

'/Users/taraperrigeold/Documents/Documents - Tara Perrige’s MacBook Pro/CareerFoundry/Instacart Basket Analysis'

In [4]:
# Create list of columns to import from orders_checked.csv
vars_list = ['order_id', 'user_id', 'order_number', 'orders_day_of_week', 'order_hour_of_day', 'days_since_prior_order']

In [5]:
# Check the output
vars_list

['order_id',
 'user_id',
 'order_number',
 'orders_day_of_week',
 'order_hour_of_day',
 'days_since_prior_order']

In [6]:
# Import orders_checked.csv
df_ords = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'orders_checked.csv'), usecols = vars_list)

In [7]:
# Check the output
df_ords.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order
0,2539329,1,1,2,8,NaN
1,2398795,1,2,3,7,15.0
2,473747,1,3,3,12,21.0
3,2254736,1,4,4,7,29.0
4,431534,1,5,4,15,28.0


In [9]:
# Import order_products_prior.csv
df_ords_prior = pd.read_csv(os.path.join(path, '02 Data', 'Original Data', 'order_products_prior.csv'), index_col = False)

In [10]:
# Check the output
df_ords_prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [11]:
# Check dimensions of df_ords
df_ords.shape

(3421083, 6)

In [12]:
# Check dimensions of df_ords_prior
df_ords_prior.shape

(32434489, 4)

# 03. Data consistency checks - df_ords_prior

## 01. Mixed-type values

In [13]:
# Check for mixed types
for col in df_ords_prior.columns.tolist():
    weird = (df_ords_prior[[col]].applymap(type) != df_ords_prior[[col]].iloc[0].apply(type)).any(axis = 1)
    if len (df_ords_prior[weird]) > 0:
        print(col)

No mixed-type values found.

## 02. Missing values

In [14]:
# Find missing values in df_ords_prior
df_ords_prior.isnull().sum()

order_id             0
product_id           0
add_to_cart_order    0
reordered            0
dtype: int64

No missing values found.

## 03. Duplicate values

In [15]:
# Check for full duplicates by creating a subset of only rows that are duplicates
df_dups = df_ords_prior[df_ords_prior.duplicated()]

In [16]:
# Check output
df_dups.head()

,order_id,product_id,add_to_cart_order,reordered


No full duplicates found.

# 04. Merging df_ords and df_ords_prior

In [12]:
# Merge df_ords and df_ords_prior using order_id as a key, add a merge flag
df_merged_large = df_ords.merge(df_ords_prior, on = 'order_id', indicator = True)

In [13]:
# Check the output
df_merged_large.head()

,order_id,user_id,order_number,orders_day_of_week,order_hour_of_day,days_since_prior_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,196,1,0,both
1,2539329,1,1,2,8,NaN,14084,2,0,both
2,2539329,1,1,2,8,NaN,12427,3,0,both
3,2539329,1,1,2,8,NaN,26088,4,0,both
4,2539329,1,1,2,8,NaN,26405,5,0,both


In [14]:
# Check merge flag frequency
df_merged_large['_merge'].value_counts()

both          32434489
left_only            0
right_only           0
Name: _merge, dtype: int64

# 05. Exporting df_merged_large

In [15]:
# Export df_merged_large in pkl format
df_merged_large.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))